# ART Data Preparation and Cleaning

This notebook loads the raw ART pretest data, runs diagnostics, applies cleaning decisions,
and saves the cleaned dataset for downstream IRT analysis.

**Output:** `data/processed/art_cleaned/`

**Documentation:** `docs/art_cleaning/CLEANING_DECISIONS.md`

## Section 0: Setup and Configuration

Import libraries, set paths, and convert Excel files to CSV.

In [1]:
# ============================================================
# IMPORTS
# ============================================================
import pandas as pd
import numpy as np
from pathlib import Path
import re
from datetime import datetime

print("Imports loaded.")
print(f"Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Imports loaded.
Started: 2026-02-16 17:36:55


In [2]:
# ============================================================
# CONFIGURATION
# ============================================================
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

PROJECT_ROOT = Path("../..")
DATA_DIR = PROJECT_ROOT / "data" / "raw"
OUTPUT_DIR = PROJECT_ROOT / "data" / "processed" / "art_cleaned"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

ART_FILE = DATA_DIR / "ART_pretest_merged_EN.xlsx"
REAL_AUTHORS_FILE = DATA_DIR / "author_lists" / "real_authors.xls"
FOILS_FILE = DATA_DIR / "author_lists" / "not_real_authors.xls"

ART_CSV = DATA_DIR / "ART_pretest_merged_EN.csv"
REAL_AUTHORS_CSV = DATA_DIR / "author_lists" / "real_authors.csv"
FOILS_CSV = DATA_DIR / "author_lists" / "not_real_authors.csv"

print("Configuration loaded:")
print(f"  Project root: {PROJECT_ROOT.resolve()}")
print(f"  Output dir:   {OUTPUT_DIR.resolve()}")

Configuration loaded:
  Project root: /home/polina/Documents/Cursor_Projects/Russian Author Recognition Test Cursor
  Output dir:   /home/polina/Documents/Cursor_Projects/Russian Author Recognition Test Cursor/data/processed/art_cleaned


In [3]:
# ============================================================
# CONVERT XLSX/XLS TO CSV
# ============================================================
df_art = pd.read_excel(ART_FILE, header=None)
df_art.to_csv(ART_CSV, index=False, header=False)
print(f"Saved: {ART_CSV.name} ({df_art.shape[0]} rows x {df_art.shape[1]} cols)")

df_real = pd.read_excel(REAL_AUTHORS_FILE, header=None)
df_real.to_csv(REAL_AUTHORS_CSV, index=False, header=False)
print(f"Saved: {REAL_AUTHORS_CSV.name} ({df_real.shape[0]} rows)")

df_foils = pd.read_excel(FOILS_FILE, header=None)
df_foils.to_csv(FOILS_CSV, index=False, header=False)
print(f"Saved: {FOILS_CSV.name} ({df_foils.shape[0]} rows)")
print("All Excel files converted to CSV.")

Saved: ART_pretest_merged_EN.csv (1837 rows x 220 cols)
Saved: real_authors.csv (100 rows)
Saved: not_real_authors.csv (107 rows)
All Excel files converted to CSV.


## Section 1: Data Loading and Preprocessing

Load the merged ART dataset, identify real authors vs foils, classify items, and run diagnostics.

In [4]:
# ============================================================
# LOAD RAW DATA
# ============================================================
print("1. Loading ART pretest dataset...")
raw_df = pd.read_csv(ART_CSV, header=None)
labels_row = raw_df.iloc[0].tolist()
codes_row = raw_df.iloc[1].tolist()
data_df = raw_df.iloc[2:].reset_index(drop=True)
data_df.columns = labels_row
print(f"   Raw shape: {raw_df.shape[0]} rows x {raw_df.shape[1]} columns")
print(f"   Data rows: {len(data_df)} participants")

1. Loading ART pretest dataset...
   Raw shape: 1837 rows x 220 columns
   Data rows: 1835 participants


### Load reference lists and identify items

In [5]:
# ============================================================
# LOAD AUTHOR REFERENCE LISTS
# ============================================================
print("2. Loading author reference lists...")
real_authors_df = pd.read_excel(REAL_AUTHORS_FILE, header=None)
foils_df = pd.read_csv(FOILS_CSV, header=None)
print(f"   Real authors: {real_authors_df.shape[0]} rows")
print(f"   Foils: {foils_df.shape[0]} rows")

2. Loading author reference lists...
   Real authors: 100 rows
   Foils: 107 rows


In [6]:
# ============================================================
# IDENTIFY REAL AUTHORS VS FOILS IN DATASET
# ============================================================
print("3. Identifying item columns...")
n_demographics = 5
item_labels = labels_row[n_demographics:-1]
item_codes = codes_row[n_demographics:-1]
data_df.columns = labels_row
participant_ids = pd.Index(range(1, len(data_df) + 1), name="participant_id")
item_data = data_df[item_labels].copy()
source_col = data_df.iloc[:, -1].reset_index(drop=True)
print(f"   Item columns: {len(item_labels)}")

3. Identifying item columns...
   Item columns: 214


In [7]:
# ============================================================
# CLASSIFY ITEMS AS REAL AUTHORS OR FOILS
# ============================================================
print("4. Classifying items as real authors or foils...")

def extract_fill_code(text):
    if pd.isna(text): return None
    s = re.sub(r'fil\s+l(\d+)', r'fill\1', str(text), flags=re.I)
    m = re.search(r'fill\s*\d+', s, re.I)
    return m.group(0).replace(' ', '') if m else None

def is_foil_code(c): return bool(re.match(r'^fill\s*\d+$', str(c).strip(), re.I)) if pd.notna(c) else False
def is_real_author_code(c): return bool(re.match(r'^(mod|cla|soc)\d+$', str(c).strip(), re.I)) if pd.notna(c) else False
def normalize_name(n): return re.sub(r'\s*fill\s*\d+', '', str(n).strip(), flags=re.I).strip().lower() if pd.notna(n) else ""

foil_codes_from_ref = {extract_fill_code(n).lower() for n in foils_df.iloc[:, 0] if extract_fill_code(n)}
real_names_list = real_authors_df.iloc[:, 0].dropna().tolist()
foil_names_list = foils_df.iloc[:, 0].dropna().tolist()
real_names_normalized = set(normalize_name(n) for n in real_names_list)

real_author_indices, foil_indices = [], []
real_author_names, foil_names = [], []
for i, (label, code) in enumerate(zip(item_labels, item_codes)):
    cn = str(code).strip().lower()
    ln = normalize_name(label)
    if is_foil_code(code) or cn in foil_codes_from_ref or extract_fill_code(label):
        foil_indices.append(i); foil_names.append(label)
    elif ln in real_names_normalized or is_real_author_code(code):
        real_author_indices.append(i); real_author_names.append(label)

item_classification = pd.DataFrame({
    'item_index': range(len(item_labels)), 'item_label': item_labels, 'item_code': item_codes,
    'is_real_author': [i in real_author_indices for i in range(len(item_labels))],
    'is_foil': [i in foil_indices for i in range(len(item_labels))]
})
print(f"   Real authors: {len(real_author_indices)}, Foils: {len(foil_indices)}")

4. Classifying items as real authors or foils...
   Real authors: 106, Foils: 108


### Detailed Data Exploration — Pre-Cleaning Diagnostics

Run diagnostics before cleaning. See `docs/art_cleaning/CLEANING_DECISIONS.md` for full output and decisions.

In [8]:
# ============================================================
# PRE-CLEANING DIAGNOSTICS (summary)
# ============================================================
from collections import Counter
N_PARTICIPANTS = len(data_df)
N_ITEMS = len(item_labels)
ITEM_START, ITEM_END = n_demographics, len(labels_row) - 1
item_block = data_df.iloc[:, ITEM_START:ITEM_END].copy()
item_block.columns = range(N_ITEMS)

# Raw values audit
flat = pd.Series(item_block.values.flatten())
print("Raw value counts:", flat.value_counts(dropna=False).head(10).to_dict())
non_b = [(i, item_labels[i], item_codes[i]) for i in range(N_ITEMS)
         if set(item_block.iloc[:, i].dropna().astype(str).str.strip()) - {"0","1","0.0","1.0"}]
if non_b: print("Non-binary cols:", non_b)

# Duplicates
dup_labels = [l for l, c in Counter(item_labels).items() if c > 1]
print("Duplicate labels:", dup_labels)

# Missing
miss_pct = item_block.isnull().sum(axis=0) / N_PARTICIPANTS * 100
high_miss = [(item_labels[i], item_codes[i], miss_pct.iloc[i]) for i in range(N_ITEMS) if miss_pct.iloc[i] > 40]
print("Items >40% missing:", len(high_miss), "—", high_miss[:5])
print("Diagnostics complete. See CLEANING_DECISIONS.md for full details.")

Raw value counts: {'0': 276317, '1': 104812, nan: 11560, 'falce': 1}
Non-binary cols: [(0, 'Gerrit HoogenbuM fill1', 'fill1')]
Duplicate labels: ['Ian Fleming']
Items >40% missing: 9 — [('Vladimir Gusakov fill 101', 'fill101', np.float64(43.596730245231605)), ('Ivan Savin fill 102', 'fill102', np.float64(43.596730245231605)), ('Sergey Nikitin fill 103', 'fill103', np.float64(87.52043596730246)), ('Evgenia Serova fill 104', 'fill104', np.float64(43.596730245231605)), ('Chabo Chucky fill 105', 'fill105', np.float64(43.596730245231605))]
Diagnostics complete. See CLEANING_DECISIONS.md for full details.


## Section 2: Data Cleaning

Apply cleaning decisions (see `docs/art_cleaning/CLEANING_DECISIONS.md`):

1. Recode 'falce' → 0 in Gerrit HoogenbuM fill1  
2. Drop Ian Fleming (mod33) — duplicate, 99.9% missing  
3. Drop 8 items with >40% missing  
4. Keep items with 5–40% missing  
5. Ignore duplicate codes (metadata only)  
6. Keep near-floor/ceiling items

In [11]:
# ============================================================
# DATA CLEANING — Apply decisions
# ============================================================
# Work with positional item block to avoid duplicate-name issues
item_block = data_df.iloc[:, ITEM_START:ITEM_END].copy()
item_block.columns = range(N_ITEMS)

# Step 1: Recode 'falce' → 0 in Gerrit HoogenbuM fill1 (item position 0)
col_fill1 = 0  # first item column
ser = item_block.iloc[:, col_fill1].astype(str).str.strip()
n_falce = (ser == "falce").sum()
# Series replacement and type conversion outside of assignment prevents dtype conflict
ser = ser.replace("falce", "0")
ser = pd.to_numeric(ser, errors="coerce")
# Replace entire column with a new Series to avoid dtype conflict with string column
item_block[col_fill1] = ser.values
print(f"Step 1: Recoded {n_falce} 'falce' → 0 in Gerrit HoogenbuM fill1")

# Step 2 & 3: Items to DROP (by label, code) — Ian Fleming mod33 + 8 high-missing
ITEMS_TO_DROP = [
    ("Ian Fleming", "mod33"),           # duplicate, 99.9% missing
    ("Sergey Nikitin fill 103", "fill103"),
    ("Lyudmila Ulitskaya", "mod32"),
    ("Alexander Tvardovsky", "cla27"),
    ("Ilya Ilf", "soc15"),
    ("Ivan Savin fill 102", "fill102"),
    ("Evgenia Serova fill 104", "fill104"),
    ("Vladimir Gusakov fill 101", "fill101"),
    ("Chabo Chucky fill 105", "fill105"),
]
indices_to_drop = [i for i in range(N_ITEMS) if (item_labels[i], str(item_codes[i])) in ITEMS_TO_DROP]
indices_to_keep = [i for i in range(N_ITEMS) if i not in indices_to_drop]

print(f"Step 2–3: Dropping {len(indices_to_drop)} items: {[item_labels[i] for i in indices_to_drop]}")

# Build cleaned item block
item_block_clean = item_block.iloc[:, indices_to_keep].copy()
labels_clean = [item_labels[i] for i in indices_to_keep]
codes_clean = [item_codes[i] for i in indices_to_keep]
N_ITEMS_CLEAN = len(labels_clean)
print(f"   Retained: {N_ITEMS_CLEAN} items ({len(indices_to_keep)} real + foil)")

Step 1: Recoded 1 'falce' → 0 in Gerrit HoogenbuM fill1
Step 2–3: Dropping 9 items: ['Vladimir Gusakov fill 101', 'Ivan Savin fill 102', 'Sergey Nikitin fill 103', 'Evgenia Serova fill 104', 'Chabo Chucky fill 105', 'Ilya Ilf', 'Lyudmila Ulitskaya', 'Alexander Tvardovsky', 'Ian Fleming']
   Retained: 205 items (205 real + foil)


In [12]:
# ============================================================
# SAVE CLEANED OUTPUT
# ============================================================
# Build output: same structure as input (row 0 = labels, row 1 = codes, rows 2+ = data)
demo_cols = labels_row[:n_demographics]
source_label = labels_row[-1]

out_labels = demo_cols + labels_clean + [source_label]
out_codes = codes_row[:n_demographics] + codes_clean + [codes_row[-1]]  # source has no code, use last

# Data: demographics + cleaned items + source
data_clean = pd.concat([
    data_df.iloc[:, :n_demographics].reset_index(drop=True),
    item_block_clean.reset_index(drop=True),
    source_col.reset_index(drop=True)
], axis=1)
data_clean.columns = range(len(out_labels))

# Header rows
header_df = pd.DataFrame([out_labels, out_codes])
out_df = pd.concat([header_df, data_clean], ignore_index=True)

# Save main cleaned CSV
OUTPUT_CSV = OUTPUT_DIR / "ART_pretest_merged_EN_cleaned.csv"
out_df.to_csv(OUTPUT_CSV, index=False, header=False)
print(f"Saved: {OUTPUT_CSV.name}")
print(f"   Shape: {out_df.shape[0]} rows x {out_df.shape[1]} columns")
print(f"   Participants: {len(data_clean)}")
print(f"   Items: {N_ITEMS_CLEAN} (dropped {N_ITEMS - N_ITEMS_CLEAN})")

# Save item metadata (retained items only)
meta = pd.DataFrame({
    "item_index": range(N_ITEMS_CLEAN),
    "item_label": labels_clean,
    "item_code": codes_clean,
    "is_real_author": [i in real_author_indices for i in indices_to_keep],
    "is_foil": [i in foil_indices for i in indices_to_keep],
})
meta.to_csv(OUTPUT_DIR / "item_metadata.csv", index=False)
print(f"Saved: item_metadata.csv")

# Save excluded items log
excluded = pd.DataFrame([
    {"item_label": item_labels[i], "item_code": str(item_codes[i]), "reason": ">40% missing or duplicate"}
    for i in indices_to_drop
])
excluded.to_csv(OUTPUT_DIR / "excluded_items.csv", index=False)
print(f"Saved: excluded_items.csv")

Saved: ART_pretest_merged_EN_cleaned.csv
   Shape: 1837 rows x 211 columns
   Participants: 1835
   Items: 205 (dropped 9)
Saved: item_metadata.csv
Saved: excluded_items.csv


In [13]:
# ============================================================
# LOAD RAW DATA
# ============================================================
print("1. Loading ART pretest dataset...")
raw_df = pd.read_csv(ART_CSV, header=None)
labels_row = raw_df.iloc[0].tolist()
codes_row = raw_df.iloc[1].tolist()
data_df = raw_df.iloc[2:].reset_index(drop=True)

print(f"   Raw shape: {raw_df.shape[0]} rows x {raw_df.shape[1]} columns")
print(f"   Data rows: {len(data_df)} participants")
print(f"   First 5 labels: {labels_row[:5]}")
print(f"   Last 5 labels: {labels_row[-5:]}")

1. Loading ART pretest dataset...
   Raw shape: 1837 rows x 220 columns
   Data rows: 1835 participants
   First 5 labels: ['Submited', 'age', 'sex ', 'humanities or not', 'education and profession']
   Last 5 labels: ['Ilya Ilf', 'Lyudmila Ulitskaya', 'Alexander Tvardovsky', 'Ian Fleming', 'source']
